In [18]:
import numpy as np
import fastembed
from fastembed import TextEmbedding
from fastembed.rerank.cross_encoder import TextCrossEncoder
from qdrant_client import QdrantClient , models
import requests 

## Q1. Embedding the query

In [2]:
# Start by choosing embedding model
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████| 5/5 [01:12<00:00, 14.45s/it]


In [3]:
# Now we need to embed this query 'I just discovered the course. Can I join now?'
query = 'I just discovered the course. Can I join now?'
embedding = list(model.embed(query))[0] # NumPy Array

In [4]:
# Question Answer
embedding.min()

np.float64(-0.11726373885183883)

## Q2. Cosine similarity with another vector

In [5]:
cosine_similarity = lambda a, b: a.dot(b)

In [6]:
query = 'I just discovered the course. Can I join now?'
doc = 'Can I still join the course after the start date?'

query_embedding = list(model.embed(query))[0]
doc_embedding = list(model.embed(doc))[0]

cosine_similarity(query_embedding, doc_embedding)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [7]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [8]:
documents_embedding = list(model.embed([doc['text'] for doc in documents]))

In [9]:
scores_array = np.array([cosine_similarity(query_embedding, doc_embedding) for doc_embedding in documents_embedding])

In [10]:
scores_array.argmax()

np.int64(1)

## Q4. Ranking by cosine, version two

In [11]:
full_text_embedding = list(model.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))

In [12]:
full_text_scores = np.array([cosine_similarity(query_embedding, doc_embedding) for doc_embedding in full_text_embedding])

In [13]:
full_text_scores.argmax()

np.int64(0)

## Q5. Selecting the embedding model

In [14]:
# Figuring the min model dimension
min_dimension = min([model['dim'] for model in TextEmbedding.list_supported_models()])
min_dimension

384

## Q6. Indexing with qdrant

In [16]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [17]:
len(documents)

375

In [19]:
q_client = QdrantClient("http://localhost:6333/")

In [20]:
collection_name = "zoomcamp-rag"

q_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size = min_dimension,
        distance=models.Distance.COSINE
    )
)

True

In [21]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [29]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id = id,
        vector=models.Document(text=doc['question'] + ' ' + doc['text'] , model="BAAI/bge-small-en"),
        payload={
            'text':doc['text'],
            'section':doc['section'],
            'question':doc['question'],
            'course':doc['course'],
        }
    )
    points.append(point)
    id +=1

In [30]:
q_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████| 5/5 [04:26<00:00, 53.30s/it]


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
def search(query , limit=1):
    results = q_client.query_points(
        collection_name=collection_name,
        query=models.Document(text = query , model="BAAI/bge-small-en"),
        limit=limit,
        with_payload=True
    )
    return results

In [40]:
search('I just discovered the course. Can I join now?')

QueryResponse(points=[ScoredPoint(id=14, version=1, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])